In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 31.5 MB/s 
     |████████████████████████████████| 163 kB 49.3 MB/s 
     |████████████████████████████████| 7.6 MB 49.8 MB/s 


In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 31.3 MB/s 
     |████████████████████████████████| 212 kB 64.7 MB/s 
     |████████████████████████████████| 115 kB 19.8 MB/s 
     |████████████████████████████████| 127 kB 58.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=b719a87b8e5c5bc0e9be065dfa3852c9eb7cddace927c813b3ab1c480182b8d7
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3585875e21a3ac610348e740409c76
Successfully built rouge-score


In [4]:
from datasets import load_dataset
import pandas as pd
import numpy as np
import re
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig, pipeline
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest
from datasets import load_metric
rouge = load_metric('rouge')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  # This is added back by InteractiveShellApp.init_path()


In [5]:
dataset = load_dataset("xsum")

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

In [7]:
df = pd.DataFrame(dataset['validation'])

In [8]:
df = df.head(30)

In [9]:
df.document[0]

'The ex-Reading defender denied fraudulent trading charges relating to the Sodje Sports Foundation - a charity to raise money for Nigerian sport.\nMr Sodje, 37, is jointly charged with elder brothers Efe, 44, Bright, 50 and Stephen, 42.\nAppearing at the Old Bailey earlier, all four denied the offence.\nThe charge relates to offences which allegedly took place between 2008 and 2014.\nSam, from Kent, Efe and Bright, of Greater Manchester, and Stephen, from Bexley, are due to stand trial in July.\nThey were all released on bail.'

In [10]:
#for i in range(len(df)):
## df['document'][i] = df['document'][i].replace('\n', '')
#  df['document'][i] = re.sub(' +', ' ', df['document'][i])
 # df['summary'][i] = df['summary'][i].replace('\n', '')
 # df['summary'][i] = df['summary'][i].strip()
 # df['summary'][i] = re.sub(' +', ' ', df['document'][i])

In [11]:
# BART Model Summarization.

In [12]:
# Initialize column for Bart Summary to be stored in.
df['BartSummary'] = 0

In [13]:
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
for i in range(len(df)):
  try:
    text = df['document'][i]
    inputs = tokenizer([text], return_tensors='pt')
    summary_ids = model.generate(inputs['input_ids'], max_length=500, early_stopping=False)
    final = [tokenizer.decode(g, skip_special_tokens=True) for g in summary_ids]
    df['BartSummary'][i] = final
  except:
    print(f'Broke for id {i}')

Downloading:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [14]:
# Pipeline simple model

In [15]:
# Initialize Column for the pipeline Summary to be stored in. 
df['PipelineSummary'] = 0

In [16]:
summarizer = pipeline('summarization')
for i in range(len(df)):
  try:
    text = df['document'][i]
    df['PipelineSummary'][i] = summarizer(text, do_sample=False)
  except:
    print(f'Broke for id {i}')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Your max_length is set to 142, but you input_length is only 123. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
Your max_length is set to 142, but you input_length is only 78. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=39)
Your max_length is set to 142, but you input_length is only 71. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=35)
Your max_length is set to 142, but you input_length is only 33. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=16)
Your max_length is set to 142, but you input_length is only 114. You might consider de

In [17]:
# Pipeline meeting fine tuned Bart Based Summarizer.

In [18]:
# Initialize columns for the fine tuned model summaries to be stored in.
df['MeetingSummary'] = 0

In [19]:
summarizer_model = 'knkarthick/MEETING_SUMMARY'
summarizer = pipeline('summarization', model=summarizer_model)
for i in range(len(df)):
  try:
    text = df['document'][i]
    df['MeetingSummary'][i] = summarizer(text, do_sample=False)
  except:
    print(f'Broke for id {i}')

Downloading:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/337 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
Your max_length is set to 62, but you input_length is only 33. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=16)


In [20]:
# Pegasus Summary

In [21]:
# Spacy Extractive Summarization.

In [22]:
df['SpaceySummary'] = 0

In [23]:
stopwords = list(STOP_WORDS)
nlp = spacy.load('en_core_web_sm')
for i in range(len(df)):
  try:
    text = df['document'][i] 
    doc = nlp(text)
    tokens = [token.text for token in doc]
    #punctuation = punctuation + "\n"
    word_frequencies = {}
    for word in doc:
      if word.text.lower() not in stopwords:
        if word.text.lower() not in punctuation:
          if word.text not in word_frequencies.keys():
            word_frequencies[word.text] = 1
          else:
            word_frequencies[word.text] += 1
    max_freq = max(word_frequencies.values())
    for word in word_frequencies.keys():
      word_frequencies[word] = word_frequencies[word]/max_freq
    sentence_tokens = [sent for sent in doc.sents]
    sent_score = {}
    for sent in sentence_tokens:
      for word in sent:
        if word.text.lower() in word_frequencies.keys():
          if sent not in sent_score.keys():
            sent_score[sent] = word_frequencies[word.text.lower()]
          else:
            sent_score[sent] += word_frequencies[word.text.lower()]
    select_len = int(len(sentence_tokens)*0.3)
    text_summary = nlargest(select_len, sent_score, key = sent_score.get)
    summary = [word.text for word in text_summary]
    final = ' '.join(summary)
    df['SpaceySummary'][i] = final
  except:
    print(f'Broke for id {i}')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [24]:
for i in range(len(df)):
  df['BartSummary'][i] = df['BartSummary'][i][0].strip()
  df['PipelineSummary'][i] = df['PipelineSummary'][i][0]['summary_text'].strip()
  df['MeetingSummary'][i] = df['MeetingSummary'][i][0]['summary_text'].strip()

In [30]:
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

In [27]:
df['PegasusSummary'] = 0

In [32]:
tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-xsum')
model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum')
for i in range(len(df)):
  try:
    text = df['document'][i]
    inputs = tokenizer(text, return_tensors='pt')
    summary_ids = model.generate(**inputs)
    final = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    df['PegasusSummary'][i] = final
  except:
    print(f'Broke for id {i}')

TypeError: ignored

In [ ]:
df['PegasusSummary']

In [33]:
# Check to see is summary is different than the document (It was the same in one try and was trying to figure out why)
df['summary'] == df['document']

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
dtype: bool

In [34]:
# Evalution for the summaries using Rouge Metric.

In [35]:
# Initialize columns for Bart evaluation to be stored in. 
df['Bartrouge1acc'] = 0.0
df['Bartrouge1rec'] = 0.0
df['Bartrouge1f1'] = 0.0
df['Bartrouge2acc'] = 0.0
df['Bartrouge2rec'] = 0.0
df['Bartrouge2f1'] = 0.0
df['BartrougeLacc'] = 0.0
df['BartrougeLrec'] = 0.0
df['BartrougeLf1'] = 0.0
df['BartrougeLsumacc'] = 0.0
df['BartrougeLsumrec'] = 0.0
df['BartrougeLsumf1'] = 0.0
# Initialize columns for Pipeline evaluation to be stored in. 
df['Pipelinerouge1acc'] = 0.0
df['Pipelinerouge1rec'] = 0.0
df['Pipelinerouge1f1'] = 0.0
df['Pipelinerouge2acc'] = 0.0
df['Pipelinerouge2rec'] = 0.0
df['Pipelinerouge2f1'] = 0.0
df['PipelinerougeLacc'] = 0.0
df['PipelinerougeLrec'] = 0.0
df['PipelinerougeLf1'] = 0.0
df['PipelinerougeLsumacc'] = 0.0
df['PipelinerougeLsumrec'] = 0.0
df['PipelinerougeLsumf1'] = 0.0
# Initialize columns for Meeting fine tuned evaluation to be stored in. 
df['Meetingrouge1acc'] = 0.0
df['Meetingrouge1rec'] = 0.0
df['Meetingrouge1f1'] = 0.0
df['Meetingrouge2acc'] = 0.0
df['Meetingrouge2rec'] = 0.0
df['Meetingrouge2f1'] = 0.0
df['MeetingrougeLacc'] = 0.0
df['MeetingrougeLrec'] = 0.0
df['MeetingrougeLf1'] = 0.0
df['MeetingrougeLsumacc'] = 0.0
df['MeetingrougeLsumrec'] = 0.0
df['MeetingrougeLsumf1'] = 0.0
# Initialize columns for Spacey evaluation to be stored in. 
df['Spaceyrouge1acc'] = 0.0
df['Spaceyrouge1rec'] = 0.0
df['Spaceyrouge1f1'] = 0.0
df['Spaceyrouge2acc'] = 0.0
df['Spaceyrouge2rec'] = 0.0
df['Spaceyrouge2f1'] = 0.0
df['SpaceyrougeLacc'] = 0.0
df['SpaceyrougeLrec'] = 0.0
df['SpaceyrougeLf1'] = 0.0
df['SpaceyrougeLsumacc'] = 0.0
df['SpaceyrougeLsumrec'] = 0.0
df['SpaceyrougeLsumf1'] = 0.0
# Initialize columns for Pegasus evaluation to be stored in. 
df['Pegasusrouge1acc'] = 0.0
df['Pegasusrouge1rec'] = 0.0
df['Pegasusrouge1f1'] = 0.0
df['Pegasusrouge2acc'] = 0.0
df['Pegasusrouge2rec'] = 0.0
df['Pegasusrouge2f1'] = 0.0
df['PegasusrougeLacc'] = 0.0
df['PegasusrougeLrec'] = 0.0
df['PegasusrougeLf1'] = 0.0
df['PegasusrougeLsumacc'] = 0.0
df['PegasusrougeLsumrec'] = 0.0
df['PegasusrougeLsumf1'] = 0.0

In [36]:
for i in range(len(df)):
  pred = [df['BartSummary'][i]]
  ref = [df['summary'][i]]
  df['Bartrouge1acc'][i] = rouge.compute(predictions=pred, references=ref)['rouge1'][0][0]
  df['Bartrouge1rec'][i] = rouge.compute(predictions=pred, references=ref)['rouge1'][0][1]
  df['Bartrouge1f1'][i] = rouge.compute(predictions=pred, references=ref)['rouge1'][0][2]
  df['Bartrouge2acc'][i] = rouge.compute(predictions=pred, references=ref)['rouge2'][0][0]
  df['Bartrouge2rec'][i] = rouge.compute(predictions=pred, references=ref)['rouge2'][0][1]
  df['Bartrouge2f1'][i] = rouge.compute(predictions=pred, references=ref)['rouge2'][0][2]
  df['BartrougeLacc'][i] = rouge.compute(predictions=pred, references=ref)['rougeL'][0][0]
  df['BartrougeLrec'][i] = rouge.compute(predictions=pred, references=ref)['rougeL'][0][1]
  df['BartrougeLf1'][i] = rouge.compute(predictions=pred, references=ref)['rougeL'][0][2]
  df['BartrougeLsumacc'][i] = rouge.compute(predictions=pred, references=ref)['rougeLsum'][0][0]
  df['BartrougeLsumrec'][i] = rouge.compute(predictions=pred, references=ref)['rougeLsum'][0][1]
  df['BartrougeLsumf1'][i] = rouge.compute(predictions=pred, references=ref)['rougeLsum'][0][2]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: Se

In [37]:
# Bart Evaluation
for i in range(len(df)):
  pred = [df['BartSummary'][i]]
  ref = [df['summary'][i]]
  df['Bartrouge1acc'][i] = rouge.compute(predictions=pred, references=ref)['rouge1'][0][0]
  df['Bartrouge1rec'][i] = rouge.compute(predictions=pred, references=ref)['rouge1'][0][1]
  df['Bartrouge1f1'][i] = rouge.compute(predictions=pred, references=ref)['rouge1'][0][2]
  df['Bartrouge2acc'][i] = rouge.compute(predictions=pred, references=ref)['rouge2'][0][0]
  df['Bartrouge2rec'][i] = rouge.compute(predictions=pred, references=ref)['rouge2'][0][1]
  df['Bartrouge2f1'][i] = rouge.compute(predictions=pred, references=ref)['rouge2'][0][2]
  df['BartrougeLacc'][i] = rouge.compute(predictions=pred, references=ref)['rougeL'][0][0]
  df['BartrougeLrec'][i] = rouge.compute(predictions=pred, references=ref)['rougeL'][0][1]
  df['BartrougeLf1'][i] = rouge.compute(predictions=pred, references=ref)['rougeL'][0][2]
  df['BartrougeLsumacc'][i] = rouge.compute(predictions=pred, references=ref)['rougeLsum'][0][0]
  df['BartrougeLsumrec'][i] = rouge.compute(predictions=pred, references=ref)['rougeLsum'][0][1]
  df['BartrougeLsumf1'][i] = rouge.compute(predictions=pred, references=ref)['rougeLsum'][0][2]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A va

In [38]:
# Simple pipeline model evaluation
for i in range(len(df)):
  pred = [df['PipelineSummary'][i]]
  ref = [df['summary'][i]]
  df['Pipelinerouge1acc'][i] = rouge.compute(predictions=pred, references=ref)['rouge1'][0][0]
  df['Pipelinerouge1rec'][i] = rouge.compute(predictions=pred, references=ref)['rouge1'][0][1]
  df['Pipelinerouge1f1'][i] = rouge.compute(predictions=pred, references=ref)['rouge1'][0][2]
  df['Pipelinerouge2acc'][i] = rouge.compute(predictions=pred, references=ref)['rouge2'][0][0]
  df['Pipelinerouge2rec'][i] = rouge.compute(predictions=pred, references=ref)['rouge2'][0][1]
  df['Pipelinerouge2f1'][i] = rouge.compute(predictions=pred, references=ref)['rouge2'][0][2]
  df['PipelinerougeLacc'][i] = rouge.compute(predictions=pred, references=ref)['rougeL'][0][0]
  df['PipelinerougeLrec'][i] = rouge.compute(predictions=pred, references=ref)['rougeL'][0][1]
  df['PipelinerougeLf1'][i] = rouge.compute(predictions=pred, references=ref)['rougeL'][0][2]
  df['PipelinerougeLsumacc'][i] = rouge.compute(predictions=pred, references=ref)['rougeLsum'][0][0]
  df['PipelinerougeLsumrec'][i] = rouge.compute(predictions=pred, references=ref)['rougeLsum'][0][1]
  df['PipelinerougeLsumf1'][i] = rouge.compute(predictions=pred, references=ref)['rougeLsum'][0][2]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A va

In [39]:
# Meeting Fine Tuned Summary Evaluation
for i in range(len(df)):
  pred = [df['MeetingSummary'][i]]
  ref = [df['summary'][i]]
  df['Meetingrouge1acc'][i] = rouge.compute(predictions=pred, references=ref)['rouge1'][0][0]
  df['Meetingrouge1rec'][i] = rouge.compute(predictions=pred, references=ref)['rouge1'][0][1]
  df['Meetingrouge1f1'][i] = rouge.compute(predictions=pred, references=ref)['rouge1'][0][2]
  df['Meetingrouge2acc'][i] = rouge.compute(predictions=pred, references=ref)['rouge2'][0][0]
  df['Meetingrouge2rec'][i] = rouge.compute(predictions=pred, references=ref)['rouge2'][0][1]
  df['Meetingrouge2f1'][i] = rouge.compute(predictions=pred, references=ref)['rouge2'][0][2]
  df['MeetingrougeLacc'][i] = rouge.compute(predictions=pred, references=ref)['rougeL'][0][0]
  df['MeetingrougeLrec'][i] = rouge.compute(predictions=pred, references=ref)['rougeL'][0][1]
  df['MeetingrougeLf1'][i] = rouge.compute(predictions=pred, references=ref)['rougeL'][0][2]
  df['MeetingrougeLsumacc'][i] = rouge.compute(predictions=pred, references=ref)['rougeLsum'][0][0]
  df['MeetingrougeLsumrec'][i] = rouge.compute(predictions=pred, references=ref)['rougeLsum'][0][1]
  df['MeetingrougeLsumf1'][i] = rouge.compute(predictions=pred, references=ref)['rougeLsum'][0][2]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A va

In [40]:
# Spacey Evaluation
for i in range(len(df)):
  pred = [df['SpaceySummary'][i]]
  ref = [df['summary'][i]]
  df['Spaceyrouge1acc'][i] = rouge.compute(predictions=pred, references=ref)['rouge1'][0][0]
  df['Spaceyrouge1rec'][i] = rouge.compute(predictions=pred, references=ref)['rouge1'][0][1]
  df['Spaceyrouge1f1'][i] = rouge.compute(predictions=pred, references=ref)['rouge1'][0][2]
  df['Spaceyrouge2acc'][i] = rouge.compute(predictions=pred, references=ref)['rouge2'][0][0]
  df['Spaceyrouge2rec'][i] = rouge.compute(predictions=pred, references=ref)['rouge2'][0][1]
  df['Spaceyrouge2f1'][i] = rouge.compute(predictions=pred, references=ref)['rouge2'][0][2]
  df['SpaceyrougeLacc'][i] = rouge.compute(predictions=pred, references=ref)['rougeL'][0][0]
  df['SpaceyrougeLrec'][i] = rouge.compute(predictions=pred, references=ref)['rougeL'][0][1]
  df['SpaceyrougeLf1'][i] = rouge.compute(predictions=pred, references=ref)['rougeL'][0][2]
  df['SpaceyrougeLsumacc'][i] = rouge.compute(predictions=pred, references=ref)['rougeLsum'][0][0]
  df['SpaceyrougeLsumrec'][i] = rouge.compute(predictions=pred, references=ref)['rougeLsum'][0][1]
  df['SpaceyrougeLsumf1'][i] = rouge.compute(predictions=pred, references=ref)['rougeLsum'][0][2]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A va

In [ ]:
# Pegasus Evaluation
for i in range(len(df)):
  pred = [df['PegasusSummary'][i]]
  ref = [df['summary'][i]]
  df['Pegasusrouge1acc'][i] = rouge.compute(predictions=pred, references=ref)['rouge1'][0][0]
  df['Pegasusrouge1rec'][i] = rouge.compute(predictions=pred, references=ref)['rouge1'][0][1]
  df['Pegasusrouge1f1'][i] = rouge.compute(predictions=pred, references=ref)['rouge1'][0][2]
  df['Pegasusrouge2acc'][i] = rouge.compute(predictions=pred, references=ref)['rouge2'][0][0]
  df['Pegasusrouge2rec'][i] = rouge.compute(predictions=pred, references=ref)['rouge2'][0][1]
  df['Pegasusrouge2f1'][i] = rouge.compute(predictions=pred, references=ref)['rouge2'][0][2]
  df['PegasusrougeLacc'][i] = rouge.compute(predictions=pred, references=ref)['rougeL'][0][0]
  df['PegasusrougeLrec'][i] = rouge.compute(predictions=pred, references=ref)['rougeL'][0][1]
  df['PegasusrougeLf1'][i] = rouge.compute(predictions=pred, references=ref)['rougeL'][0][2]
  df['PegasusrougeLsumacc'][i] = rouge.compute(predictions=pred, references=ref)['rougeLsum'][0][0]
  df['PegasusrougeLsumrec'][i] = rouge.compute(predictions=pred, references=ref)['rougeLsum'][0][1]
  df['PegasusrougeLsumf1'][i] = rouge.compute(predictions=pred, references=ref)['rougeLsum'][0][2]

In [41]:
print(f"Average F1 Scores for each model:\nBart: {np.mean(df['Bartrouge1f1'])}\nPipeline: {np.mean(df['Pipelinerouge1f1'])}\nMeeting Fine Tuned: {np.mean(df['Meetingrouge1f1'])}\nPegasus: {np.mean(df['Pegasusrouge1f1'])}\nSpacey: {np.mean(df['Spaceyrouge1f1'])}")

Average F1 Scores for each model:
Bart: 0.19359367015365686
Pipeline: 0.2037653686389446
Meeting Fine Tuned: 0.3733467935775472
Pegasus: 0.0
Spacey: 0.15706597673330452


In [42]:
print(f"Average Accuracy Scores for each model:\nBart: {np.mean(df['Bartrouge1acc'])}\nPipeline: {np.mean(df['Pipelinerouge1acc'])}\nMeeting Fine Tuned: {np.mean(df['Meetingrouge1acc'])}\nPegasus: {np.mean(df['Pegasusrouge1acc'])}\nSpacey: {np.mean(df['Spaceyrouge1acc'])}")

Average Accuracy Scores for each model:
Bart: 0.1311608900922523
Pipeline: 0.14437492187523748
Meeting Fine Tuned: 0.406971287038732
Pegasus: 0.0
Spacey: 0.10977094839155184
